Goal of this notebook:

 - Implement the first type of determining connections between edges at a time:
   - Each new connection is represented by a separate edge with a start and end time.
   - When querying the graph with a specific timestamp, filter the edges based on the time and get the count. If count > 0, then a connection existed between the edges at this time.

In [7]:
# Jupyter notebook needs this or else it will crash
import nest_asyncio
nest_asyncio.apply()

from gremlin_python import statics
from gremlin_python.structure.graph import Graph
from gremlin_python.process.graph_traversal import __
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection
from gremlin_python.process.traversal import P # NEW!!! Import predicates (gt, gte, lt, lte, etc.)

# Instantiate a Gremlin Graph
graph = Graph()

# Connect to the server, instantiate traversal of graph.
g = graph.traversal().withRemote(DriverRemoteConnection('ws://localhost:8182/gremlin','g'))

# Get the vertices of the graph as a list, and print them.

print(g.V().toList())

GremlinServerError: 500: Could not execute operation due to backend exception

In [5]:
# Drop all vertices of the graph.
g.V().drop().iterate()
g.E().drop().iterate()

KeyboardInterrupt: 

In [72]:
# Add two vertices that are to be connected

# Radio frequency over fiber
g.addV().property('name', 'RFoF').next()

# Analog-Digital converter
g.addV().property('name', 'ADC').next()

# Optical fiber
g.addV().property('name', 'OF').next()


v[82170056]

In [73]:
def connected(name1: str, name2: str, time: float) -> bool:
    """
    Given two vertices labelled with <name1> and <name2>, determine whether they were connected at time <time>. 
    Do so by sending a Gremlin query to determine whether there exists an edge between the 
    two vertices such that <time> falls in between their "start_time" and "end_time" parameters.
    TODO: Add sphinx documentation if this will be implemented into the actual Python library.
    """

    # Get the vertices associated with the names
    # v1, v2 = g.V().has('name', name1).next(), g.V().has('name', name2).next()

    # Return whether there are edges that:
    #   - connect v1 and v2, 
    #   - labelled 'connection',
    #   - have a start time that is less than or equal to <time>
    #   - either do not have an end time or have an end time that is greater than or equal to <time>
    return g.V().has('name', name1).bothE('connection').as_('e').bothV().has('name', name2).select('e').and_(
            __.has('start', P.lte(time)),   # want start time to be less than or equal to <time>
            __.or_(
                __.hasNot('end'),           # end time doesn't have to exist 
                __.has('end', P.gt(time))  # OR end time must be greater than <time>
            )
        ).count().next() > 0

In [74]:
print(connected('RFoF', 'ADC', 5))

False


In [75]:
def set_connection(name1: str, name2: str, time: float, connection: bool) -> None:
    """
    Given two vertices labelled with <name1> and <name2>, create a new connection or terminate their existing connection, based on the value of <bool>. Label with time <time>.

    TODO: Add sphinx documentation if this will be implemented into the actual Python library.
    """

    if connection:
        # Add an edge labelled 'connection' with a start time of <time>
        g.V().has('name', name1).as_("a").not_( # NEGATE 
            __.bothE('connection').as_('e').bothV().has('name', name2).select('e').and_(
                __.has('start', P.lte(time)),
                __.or_(
                    __.hasNot('end'),
                    __.has('end', P.gt(time))
                )
            )
        ).V().has('name', name2).as_("b").addE('connection').from_("a").to("b").property('start', time).iterate()

    else:
        # For all edges between v1 and v2 labelled 'connection' (there should only be one) that do not have an 'end' property, create an end property of <time>.
        g.V().has('name', name1).bothE('connection').as_('e').bothV().has('name', name2).select('e').hasNot('end').property('end', time).iterate()

In [76]:
# Set some connections.

rfof_adc = [(1, True), (3, False), (4, True), (5, False), (7, True)]
rfof_of = [(2, True), (3, False), (5, True), (6, False), (9, True)]

for (time, connection) in rfof_adc:
    set_connection(name1='RFoF', name2='ADC', time=time, connection=connection)

for (time, connection) in rfof_of:
    set_connection(name1='RFoF', name2='OF', time=time, connection=connection)

In [77]:
# Go through each combination of element and see if they are connected at all times from 0 to 10.


import itertools

time_min = 0
time_max = 10

names = ['RFoF', 'ADC', 'OF']

combinations = list(itertools.combinations(names, 2))

for (name1, name2) in combinations:
    for time in range(time_min, time_max + 1):
        print(f"({name1}, {name2}) at {time}: {connected(name1, name2, time)}")

(RFoF, ADC) at 0: False
(RFoF, ADC) at 1: True
(RFoF, ADC) at 2: True
(RFoF, ADC) at 3: False
(RFoF, ADC) at 4: True
(RFoF, ADC) at 5: False
(RFoF, ADC) at 6: False
(RFoF, ADC) at 7: True
(RFoF, ADC) at 8: True
(RFoF, ADC) at 9: True
(RFoF, ADC) at 10: True
(RFoF, OF) at 0: False
(RFoF, OF) at 1: False
(RFoF, OF) at 2: True
(RFoF, OF) at 3: False
(RFoF, OF) at 4: False
(RFoF, OF) at 5: True
(RFoF, OF) at 6: False
(RFoF, OF) at 7: False
(RFoF, OF) at 8: False
(RFoF, OF) at 9: True
(RFoF, OF) at 10: True
(ADC, OF) at 0: False
(ADC, OF) at 1: False
(ADC, OF) at 2: False
(ADC, OF) at 3: False
(ADC, OF) at 4: False
(ADC, OF) at 5: False
(ADC, OF) at 6: False
(ADC, OF) at 7: False
(ADC, OF) at 8: False
(ADC, OF) at 9: False
(ADC, OF) at 10: False


In [78]:
# Go through each combination of element and see if they are connected at all times from 0 to 10.

import itertools
import datetime

time_min = 0
time_max = 10

attempts = 5

names = ['RFoF', 'ADC', 'OF']

combinations = list(itertools.combinations(names, 2))

now = datetime.datetime.now()

for _ in range(attempts):
    for (name1, name2) in combinations:
        for time in range(time_min, time_max + 1):
            connected(name1, name2, time)

seconds = (datetime.datetime.now() - now).total_seconds()

print("Average time per call:", seconds / (attempts * (time_max + 1) * len(combinations)))

Average time per call: 0.015809757575757578
